In [8]:
import time
from datetime import datetime, timedelta
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import os
import json
import random

def setup_driver():
    """Set up and return a Chrome webdriver."""
    chrome_options = Options()
    # Uncomment for headless mode if you don't need to see the browser
    # chrome_options.add_argument("--headless")
    chrome_options.add_argument("--window-size=1920,1080")
    
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    return driver

def generate_time_windows(start_date, end_date):
    """Generate time windows for each day between start_date and end_date."""
    time_windows = []
    current_date = start_date
    
    while current_date <= end_date:
        # Time windows for the day (4 per day as specified)
        windows = [
            (current_date.replace(hour=0, minute=0), current_date.replace(hour=6, minute=0)),    # 12:00AM-6:00AM
            (current_date.replace(hour=6, minute=1), current_date.replace(hour=12, minute=0)),   # 6:01AM-12:00PM
            (current_date.replace(hour=12, minute=1), current_date.replace(hour=18, minute=0)),  # 12:01PM-6:00PM
            (current_date.replace(hour=18, minute=1), current_date.replace(hour=23, minute=59))  # 6:01PM-11:59PM
        ]
        time_windows.extend(windows)
        current_date += timedelta(days=1)
    
    return time_windows

def format_datetime_for_input(dt):
    """Format datetime object to string for the input fields."""
    date_str = dt.strftime("%m/%d/%Y")
    time_str = dt.strftime("%H:%M")
    return f"{date_str}, {time_str}"

def save_checkpoint(index, time_windows):
    """Save the current progress to a checkpoint file."""
    checkpoint_dir = "checkpoints"
    os.makedirs(checkpoint_dir, exist_ok=True)
    
    start_time, end_time = time_windows[index]
    
    checkpoint = {
        "index": index,
        "start_time": start_time.isoformat(),
        "end_time": end_time.isoformat()
    }
    
    with open(f"{checkpoint_dir}/checkpoint.json", "w") as f:
        json.dump(checkpoint, f)
    
    print(f"Checkpoint saved at index {index}")

def load_checkpoint():
    """Load the last checkpoint if it exists."""
    checkpoint_file = "checkpoints/checkpoint.json"
    if os.path.exists(checkpoint_file):
        with open(checkpoint_file, "r") as f:
            checkpoint = json.load(f)
            
        index = checkpoint["index"]
        start_time = datetime.fromisoformat(checkpoint["start_time"])
        end_time = datetime.fromisoformat(checkpoint["end_time"])
        
        print(f"Resuming from checkpoint: index {index}")
        return index, (start_time, end_time)
    
    return None

def scrape_posts(driver):
    """Scrape post information from the current page."""
    try:
        # Wait for content to load
        WebDriverWait(driver, 15).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".post-wrapper, .no-results"))
        )
        
        # Check if there are no results
        no_results = driver.find_elements(By.CSS_SELECTOR, ".no-results")
        if no_results:
            print("No results found for this time window.")
            return []
        
        # Get all post wrappers
        post_wrappers = driver.find_elements(By.CSS_SELECTOR, ".post-wrapper")
        
        if not post_wrappers:
            print("No posts found on the page.")
            return []
        
        posts_data = []
        for post in post_wrappers:
            try:
                # Extract date
                date_element = post.find_element(By.CSS_SELECTOR, ".date")
                date = date_element.text.strip()
                
                # Extract username
                username_element = post.find_element(By.CSS_SELECTOR, ".author")
                username = username_element.text.strip()
                
                # Extract title
                title_element = post.find_element(By.CSS_SELECTOR, ".title")
                title = title_element.text.strip()
                
                # Extract content if available
                content = ""
                try:
                    content_elements = post.find_elements(By.CSS_SELECTOR, ".md")
                    if content_elements:
                        content = content_elements[0].text.strip()
                except:
                    # Some posts might not have content
                    pass
                
                posts_data.append({
                    "date": date,
                    "username": username,
                    "title": title,
                    "content": content
                })
            except Exception as e:
                print(f"Error extracting post data: {e}")
        
        return posts_data
    except Exception as e:
        print(f"Error scraping posts: {e}")
        return []

def save_to_csv(data, year):
    """Save data to a CSV file for the specified year."""
    if not data:
        return
    
    df = pd.DataFrame(data)
    
    # Create directory if it doesn't exist
    os.makedirs("reddit_stocks_data", exist_ok=True)
    
    # Save to CSV
    filename = f"reddit_stocks_data/stocks_{year}.csv"
    
    # If file exists, append without header
    if os.path.exists(filename):
        df.to_csv(filename, mode='a', header=False, index=False)
    else:
        df.to_csv(filename, index=False)
    
    print(f"Saved {len(data)} posts to {filename}")

def main():
    start_date = datetime(2014, 1, 1)
    # end_date = datetime(2025, 3, 1)
    end_date = datetime(2014, 1, 3)
    
    # Generate time windows
    time_windows = generate_time_windows(start_date, end_date)
    print(f"Generated {len(time_windows)} time windows.")
    
    # Check if there's a checkpoint to resume from
    checkpoint = load_checkpoint()
    start_index = 0
    
    if checkpoint:
        checkpoint_index, (checkpoint_start, checkpoint_end) = checkpoint
        
        # Find the matching time window to resume from
        for i, (start, end) in enumerate(time_windows):
            if (start == checkpoint_start and end == checkpoint_end):
                start_index = i + 1  # Start from the next window
                print(f"Resuming from checkpoint: {start_index}/{len(time_windows)}")
                break
    
    driver = setup_driver()
    
    try:
        # Navigate to the search page
        driver.get("https://search.pullpush.io/?kind=submission&subreddit=stocks&size=100")
        
        # Wait for page to load
        time.sleep(5)
        
        for i, (start_time, end_time) in enumerate(time_windows[start_index:], start=start_index):
            year = start_time.year
            print(f"Processing window {i+1}/{len(time_windows)}: {start_time} to {end_time}")
            
            try:
                # Set subreddit (should already be filled with "stocks" from URL)
                subreddit_field = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "input[placeholder='e.g. Daleeburg']"))
                )
                subreddit_field.clear()
                subreddit_field.send_keys("stocks")
                
                # Set number of results to 100
                num_field = driver.find_element(By.CSS_SELECTOR, "input[placeholder='100']")
                num_field.clear()
                num_field.send_keys("100")
                
                # Find and fill date fields
                date_fields = driver.find_elements(By.CSS_SELECTOR, "input[placeholder='mm/dd/yyyy, --:-- --']")
                
                if len(date_fields) >= 2:
                    # Set since date/time
                    since_field = date_fields[0]
                    since_field.clear()
                    since_field.send_keys(format_datetime_for_input(start_time))
                    
                    # Set until date/time
                    until_field = date_fields[1]
                    until_field.clear()
                    until_field.send_keys(format_datetime_for_input(end_time))
                else:
                    print("Could not find date input fields.")
                    continue
                
                # Click search button
                search_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, "button.search-button"))
                )
                search_button.click()
                
                # Wait for search to complete (with random delay to avoid detection)
                time.sleep(random.uniform(5, 8))
                
                # Scrape the posts
                posts_data = scrape_posts(driver)
                
                # Save data to CSV
                save_to_csv(posts_data, year)
                
                # Save checkpoint
                save_checkpoint(i, time_windows)
                
                # Random delay between requests
                time.sleep(random.uniform(2, 4))
            
            except Exception as e:
                print(f"Error processing time window: {e}")
                # Take a screenshot for debugging
                driver.save_screenshot(f"error_{i}.png")
                # Continue with the next time window
                continue
    
    finally:
        driver.quit()

if __name__ == "__main__":
    main()

Generated 12 time windows.
Processing window 1/12: 2014-01-01 00:00:00 to 2014-01-01 06:00:00
Could not find date input fields.
Processing window 2/12: 2014-01-01 06:01:00 to 2014-01-01 12:00:00
Could not find date input fields.
Processing window 3/12: 2014-01-01 12:01:00 to 2014-01-01 18:00:00
Could not find date input fields.
Processing window 4/12: 2014-01-01 18:01:00 to 2014-01-01 23:59:00
Could not find date input fields.
Processing window 5/12: 2014-01-02 00:00:00 to 2014-01-02 06:00:00
Could not find date input fields.
Processing window 6/12: 2014-01-02 06:01:00 to 2014-01-02 12:00:00
Could not find date input fields.
Processing window 7/12: 2014-01-02 12:01:00 to 2014-01-02 18:00:00
Could not find date input fields.
Processing window 8/12: 2014-01-02 18:01:00 to 2014-01-02 23:59:00
Could not find date input fields.
Processing window 9/12: 2014-01-03 00:00:00 to 2014-01-03 06:00:00
Could not find date input fields.
Processing window 10/12: 2014-01-03 06:01:00 to 2014-01-03 12:00

In [ ]:
import time
from datetime import datetime, timedelta
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import os
import json
import random

def setup_driver():
    """Set up and return a Chrome webdriver."""
    chrome_options = Options()
    # Uncomment for headless mode if you don't need to see the browser
    # chrome_options.add_argument("--headless")
    chrome_options.add_argument("--window-size=1920,1080")
    
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    return driver

def generate_time_windows(start_date, end_date):
    """Generate time windows for each day between start_date and end_date."""
    time_windows = []
    current_date = start_date
    
    while current_date <= end_date:
        # Time windows for the day (4 per day as specified)
        windows = [
            (current_date.replace(hour=0, minute=0), current_date.replace(hour=6, minute=0)),    # 12:00AM-6:00AM
            (current_date.replace(hour=6, minute=1), current_date.replace(hour=12, minute=0)),   # 6:01AM-12:00PM
            (current_date.replace(hour=12, minute=1), current_date.replace(hour=18, minute=0)),  # 12:01PM-6:00PM
            (current_date.replace(hour=18, minute=1), current_date.replace(hour=23, minute=59))  # 6:01PM-11:59PM
        ]
        time_windows.extend(windows)
        current_date += timedelta(days=1)
    
    return time_windows

def format_datetime_for_input(dt):
    """Format datetime object to string for the input fields in the expected format."""
    # Format to YYYY-MM-DDThh:mm format for HTML datetime-local input
    return dt.strftime("%Y-%m-%dT%H:%M")

def save_checkpoint(index, time_windows):
    """Save the current progress to a checkpoint file."""
    checkpoint_dir = "checkpoints"
    os.makedirs(checkpoint_dir, exist_ok=True)
    
    start_time, end_time = time_windows[index]
    
    checkpoint = {
        "index": index,
        "start_time": start_time.isoformat(),
        "end_time": end_time.isoformat()
    }
    
    with open(f"{checkpoint_dir}/checkpoint.json", "w") as f:
        json.dump(checkpoint, f)
    
    print(f"Checkpoint saved at index {index}")

def load_checkpoint():
    """Load the last checkpoint if it exists."""
    checkpoint_file = "checkpoints/checkpoint.json"
    if os.path.exists(checkpoint_file):
        with open(checkpoint_file, "r") as f:
            checkpoint = json.load(f)
            
        index = checkpoint["index"]
        start_time = datetime.fromisoformat(checkpoint["start_time"])
        end_time = datetime.fromisoformat(checkpoint["end_time"])
        
        print(f"Resuming from checkpoint: index {index}")
        return index, (start_time, end_time)
    
    return None

def scrape_posts(driver):
    """Scrape post information from the current page."""
    try:
        # Wait for content to load - either posts or no results message
        WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, ".card, #no-results"))
        )
        
        # Check if there are no results
        no_results = driver.find_elements(By.ID, "no-results")
        if no_results:
            print("No results found for this time window.")
            return []
        
        # Get all post cards
        post_cards = driver.find_elements(By.CSS_SELECTOR, ".card")
        
        if not post_cards:
            print("No posts found on the page.")
            return []
        
        posts_data = []
        for card in post_cards:
            try:
                # Extract subreddit, username, and date
                field_elements = card.find_elements(By.CSS_SELECTOR, ".field span")
                subreddit = field_elements[0].text.strip() if len(field_elements) > 0 else ""
                username = field_elements[1].text.strip() if len(field_elements) > 1 else ""
                date_str = field_elements[2].text.strip() if len(field_elements) > 2 else ""
                
                # Extract title
                title_element = card.find_element(By.CSS_SELECTOR, ".reddit-title")
                title = title_element.text.strip()
                
                # Extract content
                content = ""
                try:
                    content_element = card.find_element(By.CSS_SELECTOR, ".markdown")
                    content = content_element.text.strip()
                except:
                    # Some posts might not have content
                    pass
                
                # Extract link to Reddit
                reddit_link = ""
                try:
                    link_element = card.find_element(By.CSS_SELECTOR, "a.is-pulled-right")
                    reddit_link = link_element.get_attribute("href")
                except:
                    # Some posts might not have a link
                    pass
                
                posts_data.append({
                    "subreddit": subreddit,
                    "username": username,
                    "date": date_str,
                    "title": title,
                    "content": content,
                    "reddit_link": reddit_link
                })
            except Exception as e:
                print(f"Error extracting post data: {e}")
        
        return posts_data
    except Exception as e:
        print(f"Error scraping posts: {e}")
        return []

def save_to_csv(data, year):
    """Save data to a CSV file for the specified year."""
    if not data:
        return
    
    df = pd.DataFrame(data)
    
    # Create directory if it doesn't exist
    os.makedirs("reddit_stocks_data", exist_ok=True)
    
    # Save to CSV
    filename = f"reddit_stocks_data/stocks_{year}.csv"
    
    # If file exists, append without header
    if os.path.exists(filename):
        df.to_csv(filename, mode='a', header=False, index=False)
    else:
        df.to_csv(filename, index=False)
    
    print(f"Saved {len(data)} posts to {filename}")

def main():
    # Configure your date range
    start_date = datetime(2015, 1, 1)
    end_date = datetime(2025, 3, 1)  # For full scrape
    # end_date = datetime(2014, 1, 3)  # For testing
    
    # Generate time windows
    time_windows = generate_time_windows(start_date, end_date)
    print(f"Generated {len(time_windows)} time windows.")
    
    # Check if there's a checkpoint to resume from
    checkpoint = load_checkpoint()
    start_index = 0
    
    if checkpoint:
        checkpoint_index, (checkpoint_start, checkpoint_end) = checkpoint
        
        # Find the matching time window to resume from
        for i, (start, end) in enumerate(time_windows):
            if (start == checkpoint_start and end == checkpoint_end):
                start_index = i + 1  # Start from the next window
                print(f"Resuming from checkpoint: {start_index}/{len(time_windows)}")
                break
    
    driver = setup_driver()
    
    try:
        # Navigate to the search page
        driver.get("https://search.pullpush.io/")
        
        # Wait for page to load
        time.sleep(5)
        
        for i, (start_time, end_time) in enumerate(time_windows[start_index:], start=start_index):
            year = start_time.year
            print(f"Processing window {i+1}/{len(time_windows)}: {start_time} to {end_time}")
            
            try:
                # Set subreddit
                subreddit_field = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.ID, "subreddit"))
                )
                subreddit_field.clear()
                subreddit_field.send_keys("stocks")
                
                # Set search type to submissions (posts)
                search_type = driver.find_element(By.ID, "kind")
                search_type.click()
                driver.find_element(By.CSS_SELECTOR, "option[value='submission']").click()
                
                # Set number of results to 100
                num_field = driver.find_element(By.ID, "size")
                num_field.clear()
                num_field.send_keys("100")
                
                # Set since date/time
                since_field = driver.find_element(By.ID, "since")
                since_field.clear()
                driver.execute_script(
                    "arguments[0].value = arguments[1]", 
                    since_field, 
                    format_datetime_for_input(start_time)
                )
                
                # Set until date/time
                until_field = driver.find_element(By.ID, "until")
                until_field.clear()
                driver.execute_script(
                    "arguments[0].value = arguments[1]", 
                    until_field, 
                    format_datetime_for_input(end_time)
                )
                
                # Check render markdown (should be checked by default)
                render_md = driver.find_element(By.ID, "renderMD")
                if not render_md.is_selected():
                    render_md.click()
                
                # Click search button
                search_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.ID, "searchButton"))
                )
                search_button.click()
                
                # Wait for search to complete (with random delay to avoid detection)
                time.sleep(random.uniform(5, 8))
                
                # Scrape the posts
                posts_data = scrape_posts(driver)
                
                # Save data to CSV
                save_to_csv(posts_data, year)
                
                # Save checkpoint
                save_checkpoint(i, time_windows)
                
                # Random delay between requests
                time.sleep(random.uniform(2, 4))
            
            except Exception as e:
                print(f"Error processing time window: {e}")
                # Take a screenshot for debugging
                driver.save_screenshot(f"error_{i}.png")
                # Continue with the next time window
                continue
    
    finally:
        driver.quit()

if __name__ == "__main__":
    main()

Generated 14852 time windows.
Resuming from checkpoint: index 11
Processing window 1/14852: 2015-01-01 00:00:00 to 2015-01-01 06:00:00
Error scraping posts: Message: 
Stacktrace:
0   chromedriver                        0x0000000100d21708 chromedriver + 5969672
1   chromedriver                        0x0000000100d1932a chromedriver + 5935914
2   chromedriver                        0x00000001007d5650 chromedriver + 415312
3   chromedriver                        0x0000000100826e94 chromedriver + 749204
4   chromedriver                        0x0000000100827041 chromedriver + 749633
5   chromedriver                        0x0000000100877294 chromedriver + 1077908
6   chromedriver                        0x000000010084d19d chromedriver + 905629
7   chromedriver                        0x00000001008744f7 chromedriver + 1066231
8   chromedriver                        0x000000010084cf43 chromedriver + 905027
9   chromedriver                        0x0000000100818cea chromedriver + 691434
10  chr

Saved 4 posts to reddit_stocks_data/stocks_2015.csv
Checkpoint saved at index 31
Processing window 33/14852: 2015-01-09 00:00:00 to 2015-01-09 06:00:00
Saved 3 posts to reddit_stocks_data/stocks_2015.csv
Checkpoint saved at index 32
Processing window 34/14852: 2015-01-09 06:01:00 to 2015-01-09 12:00:00
Saved 4 posts to reddit_stocks_data/stocks_2015.csv
Checkpoint saved at index 33
Processing window 35/14852: 2015-01-09 12:01:00 to 2015-01-09 18:00:00
Saved 10 posts to reddit_stocks_data/stocks_2015.csv
Checkpoint saved at index 34
Processing window 36/14852: 2015-01-09 18:01:00 to 2015-01-09 23:59:00
Saved 2 posts to reddit_stocks_data/stocks_2015.csv
Checkpoint saved at index 35
Processing window 37/14852: 2015-01-10 00:00:00 to 2015-01-10 06:00:00
Saved 1 posts to reddit_stocks_data/stocks_2015.csv
Checkpoint saved at index 36
Processing window 38/14852: 2015-01-10 06:01:00 to 2015-01-10 12:00:00
Saved 2 posts to reddit_stocks_data/stocks_2015.csv
Checkpoint saved at index 37
Proces

Saved 5 posts to reddit_stocks_data/stocks_2015.csv
Checkpoint saved at index 74
Processing window 76/14852: 2015-01-19 18:01:00 to 2015-01-19 23:59:00
Saved 4 posts to reddit_stocks_data/stocks_2015.csv
Checkpoint saved at index 75
Processing window 77/14852: 2015-01-20 00:00:00 to 2015-01-20 06:00:00
Error scraping posts: Message: 
Stacktrace:
0   chromedriver                        0x0000000100d21708 chromedriver + 5969672
1   chromedriver                        0x0000000100d1932a chromedriver + 5935914
2   chromedriver                        0x00000001007d5650 chromedriver + 415312
3   chromedriver                        0x0000000100826e94 chromedriver + 749204
4   chromedriver                        0x0000000100827041 chromedriver + 749633
5   chromedriver                        0x0000000100877294 chromedriver + 1077908
6   chromedriver                        0x000000010084d19d chromedriver + 905629
7   chromedriver                        0x00000001008744f7 chromedriver + 1066231


Processing window 106/14852: 2015-01-27 06:01:00 to 2015-01-27 12:00:00
Saved 8 posts to reddit_stocks_data/stocks_2015.csv
Checkpoint saved at index 105
Processing window 107/14852: 2015-01-27 12:01:00 to 2015-01-27 18:00:00
Saved 5 posts to reddit_stocks_data/stocks_2015.csv
Checkpoint saved at index 106
Processing window 108/14852: 2015-01-27 18:01:00 to 2015-01-27 23:59:00
Saved 9 posts to reddit_stocks_data/stocks_2015.csv
Checkpoint saved at index 107
Processing window 109/14852: 2015-01-28 00:00:00 to 2015-01-28 06:00:00
Saved 4 posts to reddit_stocks_data/stocks_2015.csv
Checkpoint saved at index 108
Processing window 110/14852: 2015-01-28 06:01:00 to 2015-01-28 12:00:00
Saved 12 posts to reddit_stocks_data/stocks_2015.csv
Checkpoint saved at index 109
Processing window 111/14852: 2015-01-28 12:01:00 to 2015-01-28 18:00:00
Saved 7 posts to reddit_stocks_data/stocks_2015.csv
Checkpoint saved at index 110
Processing window 112/14852: 2015-01-28 18:01:00 to 2015-01-28 23:59:00
Sav

Saved 3 posts to reddit_stocks_data/stocks_2015.csv
Checkpoint saved at index 136
Processing window 138/14852: 2015-02-04 06:01:00 to 2015-02-04 12:00:00
Saved 15 posts to reddit_stocks_data/stocks_2015.csv
Checkpoint saved at index 137
Processing window 139/14852: 2015-02-04 12:01:00 to 2015-02-04 18:00:00
Saved 9 posts to reddit_stocks_data/stocks_2015.csv
Checkpoint saved at index 138
Processing window 140/14852: 2015-02-04 18:01:00 to 2015-02-04 23:59:00
Saved 8 posts to reddit_stocks_data/stocks_2015.csv
Checkpoint saved at index 139
Processing window 141/14852: 2015-02-05 00:00:00 to 2015-02-05 06:00:00
Saved 3 posts to reddit_stocks_data/stocks_2015.csv
Checkpoint saved at index 140
Processing window 142/14852: 2015-02-05 06:01:00 to 2015-02-05 12:00:00
Saved 17 posts to reddit_stocks_data/stocks_2015.csv
Checkpoint saved at index 141
Processing window 143/14852: 2015-02-05 12:01:00 to 2015-02-05 18:00:00
Saved 18 posts to reddit_stocks_data/stocks_2015.csv
Checkpoint saved at i

...

Processing window 13126/14852: 2023-12-26 06:01:00 to 2023-12-26 12:00:00
Saved 26 posts to reddit_stocks_data/stocks_2023.csv
Checkpoint saved at index 13125
Processing window 13127/14852: 2023-12-26 12:01:00 to 2023-12-26 18:00:00
Saved 31 posts to reddit_stocks_data/stocks_2023.csv
Checkpoint saved at index 13126
Processing window 13128/14852: 2023-12-26 18:01:00 to 2023-12-26 23:59:00
Saved 25 posts to reddit_stocks_data/stocks_2023.csv
Checkpoint saved at index 13127
Processing window 13129/14852: 2023-12-27 00:00:00 to 2023-12-27 06:00:00
Saved 19 posts to reddit_stocks_data/stocks_2023.csv
Checkpoint saved at index 13128
Processing window 13130/14852: 2023-12-27 06:01:00 to 2023-12-27 12:00:00
Saved 39 posts to reddit_stocks_data/stocks_2023.csv
Checkpoint saved at index 13129
Processing window 13131/14852: 2023-12-27 12:01:00 to 2023-12-27 18:00:00
Saved 42 posts to reddit_stocks_data/stocks_2023.csv
Checkpoint saved at index 13130
Processing window 13132/14852: 2023-12-27 18:0

Processing window 13167/14852: 2024-01-05 12:01:00 to 2024-01-05 18:00:00
Saved 27 posts to reddit_stocks_data/stocks_2024.csv
Checkpoint saved at index 13166
Processing window 13168/14852: 2024-01-05 18:01:00 to 2024-01-05 23:59:00
Saved 20 posts to reddit_stocks_data/stocks_2024.csv
Checkpoint saved at index 13167
Processing window 13169/14852: 2024-01-06 00:00:00 to 2024-01-06 06:00:00
Saved 16 posts to reddit_stocks_data/stocks_2024.csv
Checkpoint saved at index 13168
Processing window 13170/14852: 2024-01-06 06:01:00 to 2024-01-06 12:00:00
Saved 23 posts to reddit_stocks_data/stocks_2024.csv
Checkpoint saved at index 13169
Processing window 13171/14852: 2024-01-06 12:01:00 to 2024-01-06 18:00:00
Saved 18 posts to reddit_stocks_data/stocks_2024.csv
Checkpoint saved at index 13170
Processing window 13172/14852: 2024-01-06 18:01:00 to 2024-01-06 23:59:00
Saved 17 posts to reddit_stocks_data/stocks_2024.csv
Checkpoint saved at index 13171
Processing window 13173/14852: 2024-01-07 00:0

Processing window 13208/14852: 2024-01-15 18:01:00 to 2024-01-15 23:59:00
Saved 16 posts to reddit_stocks_data/stocks_2024.csv
Checkpoint saved at index 13207
Processing window 13209/14852: 2024-01-16 00:00:00 to 2024-01-16 06:00:00
Saved 14 posts to reddit_stocks_data/stocks_2024.csv
Checkpoint saved at index 13208
Processing window 13210/14852: 2024-01-16 06:01:00 to 2024-01-16 12:00:00
Saved 49 posts to reddit_stocks_data/stocks_2024.csv
Checkpoint saved at index 13209
Processing window 13211/14852: 2024-01-16 12:01:00 to 2024-01-16 18:00:00
Saved 39 posts to reddit_stocks_data/stocks_2024.csv
Checkpoint saved at index 13210
Processing window 13212/14852: 2024-01-16 18:01:00 to 2024-01-16 23:59:00
Saved 26 posts to reddit_stocks_data/stocks_2024.csv
Checkpoint saved at index 13211
Processing window 13213/14852: 2024-01-17 00:00:00 to 2024-01-17 06:00:00
Saved 24 posts to reddit_stocks_data/stocks_2024.csv
Checkpoint saved at index 13212
Processing window 13214/14852: 2024-01-17 06:0

Processing window 13249/14852: 2024-01-26 00:00:00 to 2024-01-26 06:00:00
Saved 21 posts to reddit_stocks_data/stocks_2024.csv
Checkpoint saved at index 13248
Processing window 13250/14852: 2024-01-26 06:01:00 to 2024-01-26 12:00:00
Saved 36 posts to reddit_stocks_data/stocks_2024.csv
Checkpoint saved at index 13249
Processing window 13251/14852: 2024-01-26 12:01:00 to 2024-01-26 18:00:00
Saved 29 posts to reddit_stocks_data/stocks_2024.csv
Checkpoint saved at index 13250
Processing window 13252/14852: 2024-01-26 18:01:00 to 2024-01-26 23:59:00
Saved 30 posts to reddit_stocks_data/stocks_2024.csv
Checkpoint saved at index 13251
Processing window 13253/14852: 2024-01-27 00:00:00 to 2024-01-27 06:00:00
Saved 18 posts to reddit_stocks_data/stocks_2024.csv
Checkpoint saved at index 13252
Processing window 13254/14852: 2024-01-27 06:01:00 to 2024-01-27 12:00:00
Saved 21 posts to reddit_stocks_data/stocks_2024.csv
Checkpoint saved at index 13253
Processing window 13255/14852: 2024-01-27 12:0

Processing window 13294/14852: 2024-02-06 06:01:00 to 2024-02-06 12:00:00
Saved 42 posts to reddit_stocks_data/stocks_2024.csv
Checkpoint saved at index 13293
Processing window 13295/14852: 2024-02-06 12:01:00 to 2024-02-06 18:00:00
Saved 62 posts to reddit_stocks_data/stocks_2024.csv
Checkpoint saved at index 13294
Processing window 13296/14852: 2024-02-06 18:01:00 to 2024-02-06 23:59:00
Saved 25 posts to reddit_stocks_data/stocks_2024.csv
Checkpoint saved at index 13295
Processing window 13297/14852: 2024-02-07 00:00:00 to 2024-02-07 06:00:00
Saved 26 posts to reddit_stocks_data/stocks_2024.csv
Checkpoint saved at index 13296
Processing window 13298/14852: 2024-02-07 06:01:00 to 2024-02-07 12:00:00
Saved 45 posts to reddit_stocks_data/stocks_2024.csv
Checkpoint saved at index 13297
Processing window 13299/14852: 2024-02-07 12:01:00 to 2024-02-07 18:00:00
Saved 72 posts to reddit_stocks_data/stocks_2024.csv
Checkpoint saved at index 13298
Processing window 13300/14852: 2024-02-07 18:0

# Test

In [2]:
!git clone https://github.com/datavorous/YARS.git

Cloning into 'YARS'...
remote: Enumerating objects: 260, done.
remote: Counting objects: 100% (112/112), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 260 (delta 69), reused 48 (delta 30), pack-reused 148 (from 1)
Receiving objects: 100% (260/260), 1.30 MiB | 1.60 MiB/s, done.
Resolving deltas: 100% (124/124), done.


In [3]:
!pip install uv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 11.0 MB/s eta 0:00:0000:0100:01


In [6]:
from yars import YARS
from utils import display_results, download_image

miner = YARS()

# Search for posts related to "OpenAI"
search_results = miner.search_reddit("OpenAI", limit=3)
display_results(search_results, "SEARCH")

# Scrape post details using its permalink
permalink = "https://www.reddit.com/r/getdisciplined/comments/1frb5ib/what_single_health_test_or_practice_has/".split('reddit.com')[1]
post_details = miner.scrape_post_details(permalink)
if post_details:
    display_results(post_details, "POST DATA")
else:
    print("Failed to scrape post details.")

# Fetch recent activity of user "iamsecb"
user_data = miner.scrape_user_data("iamsecb", limit=2)
display_results(user_data, "USER DATA")

# Fetch top posts from the subreddit "generative" from the past week
subreddit_posts = miner.fetch_subreddit_posts("generative", limit=11, category="top", time_filter="week")
display_results(subreddit_posts, "EarthPorn SUBREDDIT New Posts")

# Download images from the fetched posts
for z in range(3):
    try:
        image_url = subreddit_posts[z]["image_url"]
    except:
        image_url = subreddit_posts[z]["thumbnail_url"]
    download_image(image_url)

ModuleNotFoundError: No module named 'yars'